In [17]:
#read intput.txt
lines = []
with open('input.txt') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines]


In [18]:
#Part 1

import networkx as nx
import matplotlib.pyplot as plt
from functools import reduce
import operator

# create a graph from the input
def toGraph(lines):
    G = nx.Graph()
    for i, line in enumerate(lines):
        cache = ""
        for j, c in enumerate(line):
            node = (i,j)
            if (c != "."):
                G.add_node(node, value=c)
                if(i > 0):
                    if c in ["|", "L", "J", "S"]:
                        # get the value of the node left of this one
                        above_node_value = G.nodes.get((i-1, j), {}).get('value', "")
                        if above_node_value in ["|", "F", "7","S"]:
                            # connect the two nodes
                            G.add_edge(node, (i-1,j))
                if(j > 0):
                    if c in ["-", "7", "J", "S"]:
                        # get the value of the node left of this one
                        left_node_value = G.nodes.get((i, j-1), {}).get('value', "")
                        if left_node_value in ["-", "F", "L", "S"]:
                            # connect the two nodes
                            G.add_edge(node, (i,j-1))
    return G       

def cleanGraph(G):
    # remove all nodes that have less than 2 edges
    nodes_to_remove = [node for node in G.nodes if G.degree[node] < 2]
    for node in nodes_to_remove:
        G.remove_node(node)

    return G

def solve_part1(G):
    # find the node with the longest distance from the node with value "S" and retun the distance not the node
    start = [node for node in G.nodes if G.nodes[node].get('value', "") == "S"][0]
    end = max(nx.shortest_path_length(G, start).items(), key=operator.itemgetter(1))[1]

    print("part1:",end)

G = toGraph(lines)
G = cleanGraph(G)
solve_part1(G)

# For visualisation
def plotGraph(graph):
    plt.figure(figsize=(5, 5))  # Create a new figure with a specific size
    pos = {(i,j): (j,-i) for i, j in graph.nodes()}
    nx.draw(graph, pos, with_labels=False)  # Draw `graph`, not `G`
    labels = nx.get_node_attributes(graph, 'value')  # Get labels from `graph`, not `G`
    nx.draw_networkx_labels(graph, pos, labels=labels)  # Draw labels on `graph`, not `G`
    plt.savefig('graph.png')  # Save the figure to a file
    plt.show()

# plotGraph(G)

part1: 6870


In [19]:
#Part 2

import networkx as nx
import matplotlib.pyplot as plt
from functools import reduce
import operator

# create a graph from the input
def toGraph(lines):
    G = nx.Graph()
    for i, line in enumerate(lines):
        for j, c in enumerate(line):
            node = (i,j)
            if (c != "."):
                G.add_node(node, value=c)
                if(i > 0):
                    if c in ["|", "L", "J", "S"]:
                        # get the value of the node left of this one
                        above_node_value = G.nodes.get((i-1, j), {}).get('value', "")
                        if above_node_value in ["|", "F", "7","S"]:
                            # connect the two nodes
                            G.add_edge(node, (i-1,j))
                if(j > 0):
                    if c in ["-", "7", "J", "S"]:
                        # get the value of the node left of this one
                        left_node_value = G.nodes.get((i, j-1), {}).get('value', "")
                        if left_node_value in ["-", "F", "L", "S"]:
                            # connect the two nodes
                            G.add_edge(node, (i,j-1))
    return G       

def get_nodes_in_path(G):
    # find all nodes that are in the path starting with node with value "S"
    start = [node for node in G.nodes if G.nodes[node].get('value', "") == "S"][0]
    nodes_in_path = nx.shortest_path(G, start)
    # todo replace value S with the right value depending on the neighbours
    up = nodes_in_path.get((start[0]-1, start[1]), {})
    down = nodes_in_path.get((start[0]+1, start[1]), {})
    left = nodes_in_path.get((start[0], start[1]-1), {})
    right = nodes_in_path.get((start[0], start[1]+1), {})
    if up != {} and down != {}:
        nodes_in_path[(start[0], start[1])] = {'value': "|"}
    elif left != {} and right != {}:
        nodes_in_path[(start[0], start[1])] = {'value': "-"}
    elif up != {} and right != {}:
        nodes_in_path[(start[0], start[1])] = {'value': "L"}
    elif up != {} and left != {}:
        nodes_in_path[(start[0], start[1])] = {'value': "J"}
    elif down != {} and right != {}:
        nodes_in_path[(start[0], start[1])] = {'value': "F"}
    elif down != {} and left != {}:
        nodes_in_path[(start[0], start[1])] = {'value': "7"}

    nodes = {}
    # covert nodes_in_path to a dict with key = node and value = value
    for node in nodes_in_path:
        nodes[node] = G.nodes.get(node, {}).get('value', "")
    
    return nodes

def number_of_intescetions_with_path(path,i,j):
    #filter nodes from path that have the same j but i samller than parameter i
    nodes_with_same_j = []
    stack = []
    count = 0
    for x in reversed(range(i)):
        if (x,j) in path.keys():
            node_value = path[(x,j)]
            stack.append(node_value)
            if(node_value == "-"): # - is counted always as intersection
                count += 1
                stack = []
            else:
                first_in_stack = stack[0]
                last_in_stack = stack[-1]
                if(first_in_stack == "L" and last_in_stack == "F") or (first_in_stack == "J" and last_in_stack == "7"): # ignore this part of the path
                    stack = []
                elif(first_in_stack == "L" and last_in_stack == "7") or (first_in_stack == "J" and last_in_stack == "F"): # intersection
                    count += 1
                    stack = []
    return count

def solve_part2(G):
    nodes_in_path = get_nodes_in_path(G)
    nodes_inside_path = []
    nodes_outside_path = []
    for i, line in enumerate(lines):
        for j, c in enumerate(line):
            node = (i,j)
            if node not in nodes_in_path.keys():
                count_intersections = number_of_intescetions_with_path(nodes_in_path, i, j)
                if count_intersections > 0 and count_intersections % 2 == 1:
                    nodes_inside_path.append(node) 
                else:
                    nodes_outside_path.append(node)   

    print("part2:", len(nodes_inside_path))

G = toGraph(lines)
solve_part2(G)


# For visualisation
def plotGraph(graph):
    plt.figure(figsize=(10, 5))  # Create a new figure with a specific size
    pos = {(i,j): (j,-i) for i, j in graph.nodes()}
    nx.draw(graph, pos, with_labels=False)  # Draw `graph`, not `G`
    labels = nx.get_node_attributes(graph, 'value')  # Get labels from `graph`, not `G`
    nx.draw_networkx_labels(graph, pos, labels=labels)  # Draw labels on `graph`, not `G`
    plt.savefig('graph.png')  # Save the figure to a file
    plt.show()


# plotGraph(G)

part2: 287
